In [13]:
from py.race_function import scrape_current_race
from catboost import CatBoostRanker
import pandas as pd

In [24]:
def prediction(race_url, odds_url):
    model = CatBoostRanker()
    model.load_model('model/catboost_ranking_model.cbm')

    df = scrape_current_race(race_url, odds_url)
    X = df.iloc[:, :-1]
    predictions = model.predict(X)

    rated_scores = pd.DataFrame(predictions, columns = ['score'])

    name = df.iloc[:, -1]

    # Combine horse names and predicted scores into one DataFrame
    predicted = pd.concat([name.reset_index(drop=True), rated_scores.reset_index(drop=True)], axis=1)

    # Optionally rename columns if name column has no name
    predicted.columns = ['Horse_name', 'Predicted_Score']

    predicted = predicted.sort_values('Predicted_Score', ascending=False)

    return predicted

In [25]:
race_url = 'https://racing.hkjc.com/racing/information/English/racing/RaceCard.aspx'
odds_url = 'https://bet.hkjc.com/en/racing/wp/2025-09-07/ST/1'

df = prediction(race_url, odds_url)

In [26]:
df

,Horse_name,Predicted_Score
11,YOU'REMYEVERYTHING,0.847544
8,SPEEDY SMARTIE,0.844958
10,SUPER ELITE,0.623335
13,SONIC BOOM,0.507944
9,THE CONCENTRATION,0.468728
12,FAIRY HORSE,0.360036
7,MANYTHANKS FOREVER,0.349407
2,NOBLE DELUXE,0.147625
3,ORIENTAL SURPRISE,-0.520326
4,SUNNY DARLING,-0.673771
